In [ ]:
import os
import json
import asyncio
from src.rpc_provider import Web3AsyncRouter, AsyncRpcClient, AsyncRpcScheduler, RpcPool, RpcErrorResult
from src.logging import log
from src.web3_utils import current_utctime, normalize_block

# -----------------------------
# Environment Variables
# -----------------------------

CHAIN = os.getenv("CHAIN", "bsc").lower()
RESUME_FROM_LAST = os.getenv("RESUME_FROM_LAST", "True").lower() in ("1", "true", "yes")

# -----------------------------
# Job Name & Kafka IDs
# -----------------------------
if RESUME_FROM_LAST:
    JOB_NAME = f"{CHAIN}_backfill_resume"        # 固定名，Kafka checkpoint 能被复用
else:
    JOB_NAME = f"{CHAIN}_realtime_{current_utctime()}"  # 每次唯一，从最新block开始

TRANSACTIONAL_ID = f"blockchain.ingestion.{CHAIN}.{current_utctime()}" # TRANSACTIONAL_ID每次不一样，EOS由Compact State Topic实现

# -----------------------------
# RPC Config
# -----------------------------
RPC_CONFIG_PATH = "/etc/ingestion/rpc_providers.json"
BATCH_WINDOW_SIZE = int(os.getenv("BATCH_WINDOW_SIZE", "10")) # 一次性向 scheduler 注入多少请求（burst 注入速率）;max_queue 决定 能不能接得住
RPC_MAX_TIMEOUT = int(os.getenv("RPC_MAX_TIMEOUT", "10"))
RPC_MAX_INFLIGHT = int(os.getenv("RPC_MAX_INFLIGHT", "5"))  # 并发数量;决定 能同时飞多少个 RPC
rpc_configs = json.load(open(RPC_CONFIG_PATH))

# -----------------------------
# Init rpc pool
# -----------------------------
rpc_pool = RpcPool.from_config(rpc_configs, CHAIN)

ModuleNotFoundError: No module named 'src'

In [ ]:
async def fetch_and_push():
    client = AsyncRpcClient(timeout=RPC_MAX_TIMEOUT)
    router = Web3AsyncRouter(rpc_pool, client)
    scheduler = AsyncRpcScheduler(
        router=router,
        max_workers = 1,
        max_inflight=RPC_MAX_INFLIGHT,  #🔥 真正并发 RPC 数 # max_inflight ≈ (provider 数 × key 数) × 0.6
        max_queue=RPC_MAX_INFLIGHT * 5,
    )
    
    tasks = []
    
    for _ in range(BATCH_WINDOW_SIZE):
        tasks.append(
            asyncio.create_task(
                scheduler.submit("eth_blockNumber", [])
            )
        )
    
    results = await asyncio.gather(*tasks)

    for r in results:
        if isinstance(r, RpcErrorResult):
            log.warning(
                "rpc_failed",
                extra={
                    "provider": r.rpc,
                    "key": r.key_env,
                    "task_id": r.meta.task_id,
                    "worker": r.wid,
                    "error": type(r.error).__name__,
                },
            )
            continue
    
        result, rpc, key_env, trace, _, meta = r
        block_num = normalize_block(result)
        
    await scheduler.close()

In [ ]:
# worker as rpc executor 
# await fetch_and_push()

# rpc_call_done 是“RPC 实际完成时间线”
# rpc_call_complete 是“在 await gather 之后，按 task 列表顺序打印的”